---

## Turing Machine program for a simple non-CFL

<pre>
CSC 427, Semester 252
Burton Rosenberg
University of Miami
17 march 2025

Copyright 2025 (c) Burton Rosenberg. All rights reserved.
</pre>

---


### The Turing Machine Simulator

This page gives a Turing Machine for the non-CFL $a^ib^ic^i$. 

This is given as a text description 
of a Turing Machine that recognizes the languge using a syntax for Turing Machines description 
presented the next few paragraphs. You can read the complete 
[Syntax](https://github.com/csc427-242/sketches/blob/main/TM_Syntax.md)  and study
more [examples](https://github.com/csc427-242/sketches/blob/main/turing_machine_sketch.ipynb).

The text description is given to a Python program that carries out the computation according to the
described machine. It is a Turing Machine simulator. (That a computer, itself a form of Turing Machine,
can simulate another Turing Machine, in fact, can simulate any Turing Machine, including itself, can
be counter-intuitive.) 


*N.B.* The simulator can simluate a k-tape TM. We only describe the syntax for a one tape machine.


**Simulator Syntax**

The description is comprised of a sequence of stanzas. 
Each stanza is introduced by a stanza head comprised of keyword at the start of the line followed
by a colon. Stanzas that comprises mutliple lines have subsequent lines indented.

The stanza continues until a blank line or the start of the next stanza.

The *start, accept* and *reject* keywords are followed immediately by a state name (respectively the start, accept
and reject states.) The *accept* and *reject* stanzas can be continued on following lines to name additional accept and rejects states. 

The *state* keyword is followed by the from-state after the colon. 
The following lines in the stanza give transitions using the format,

<pre>
    read-symbol write-symbol action to-state 
</pre>

where the available actions are <verb>l, r, n</verb> (respectively move the tape head left, right or no movement). If the 
action letter is capitalized, the configuration will be printed after the use of the transition 
(useful for debugging).

**Tape symbols and wild cards**

The read and write symbols can have a wildcard using the colon "<verb>:</verb>". 

- When the wildcard is the read-symbol, it matches any character. 
- When the wildcard is the write-symbol, it equals the read-character matched.
- Wildcard matches have lower priority than matches to a specific read-symbol. 

Other special characters to watch for:

- The underscore character "<verb>_</verb>" matches the space.
- The hash character "<verb>#</verb>" anywhere on the line introduces a comment that continues to the end of the line. The hash cannot be used as a tape symbol.

**Simulation results**

The TM simulation ends when an accept or reject state is entered, when there is no matching transition rule, 
or when it reaches a maximum allowed step count.
In computation value, accept is True and the other cases are False.
Among the false cases, any case except reject is "exceptional".
The simulator can be  queried at the end of the computation which of the cases hold, 

- The machine accepts.
- The machine rejects by entering the reject state, and there is no exception.
- The machine rejects because no matching rule was found, and the exception is set to indicate this.
- The machine rejects because the simulation reached the provided maximum step count, and the exception is set to indicate this.

### Example TM description

The following example demonstrates the simulator and the syntax for the simple regular language $a^*b^*c^*$.

In [1]:
from turing_machine_sim import *


tm_abc= """# a TM program to recognize a*b*c*
# mar 6, 2025

accept: A
reject: R
start: q0

state: q0
    _ : n A
    a : r q0  # match a
    b : r q1
    c : r q2
    
state: q1
    b : r q1  # match b
    c : r q2
    _ : n A

state: q2
    c : r q2  # match c
    _ : n A

"""

# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100


tm = MachineParser.create_from_description(tm_abc)
for w in ['','a','bb', 'acc', 'abc']:
    print(f'\nstring: |{w}|')
    tm.compute_tm(w, max_steps, verbose=verbose)
    
for w in ['ba','acb', 'cb', 'aabbca' ]:
    print(f'\nstring: |{w}|')
    tm.compute_tm(w, max_steps, verbose=verbose)
    



string: ||
0 [q0]	[_]
1 [A]	[_]
accept (ok)

string: |a|
0 [q0]	[a]_
1 [q0]	a[_]
2 [A]	a[_]
accept (ok)

string: |bb|
0 [q0]	[b]b_
1 [q1]	b[b]_
2 [q1]	bb[_]
3 [A]	bb[_]
accept (ok)

string: |acc|
0 [q0]	[a]cc_
1 [q0]	a[c]c_
2 [q2]	ac[c]_
3 [q2]	acc[_]
4 [A]	acc[_]
accept (ok)

string: |abc|
0 [q0]	[a]bc_
1 [q0]	a[b]c_
2 [q1]	ab[c]_
3 [q2]	abc[_]
4 [A]	abc[_]
accept (ok)

string: |ba|
0 [q0]	[b]a_
1 [q1]	b[a]_
reject (transition missing)

string: |acb|
0 [q0]	[a]cb_
1 [q0]	a[c]b_
2 [q2]	ac[b]_
reject (transition missing)

string: |cb|
0 [q0]	[c]b_
1 [q2]	c[b]_
reject (transition missing)

string: |aabbca|
0 [q0]	[a]abbca_
1 [q0]	a[a]bbca_
2 [q0]	aa[b]bca_
3 [q1]	aab[b]ca_
4 [q1]	aabb[c]a_
5 [q2]	aabbc[a]_
reject (transition missing)



### Recognizing $a^ib^ic^i$

The language of strings $a^ib^ic^i$ is not context free. But we can recognize it with a turning machine.

*N.B.* To *recognize a language* is to accept strings in the language, and reject or undecide on strings
not in the language. Our program goes beyond this in that it *decides the language* in that it accepts
strings in the language and rejects strings not in the language. There is no non-halting case.


**Loop invariant**

For the method of loop invariants, see these
[CS 2112 Fall 2020 notes](https://www.cs.cornell.edu/courses/cs2112/2020fa/lectures/lecture.html?id=loopinv)
from Andrew Myers at Cornell University.

The algorithm depends on the loop invariant, that the string is of the form,

$$
\dashv \, \$x^*a^ix^*b^jx^*c^k
$$

If entering the loop, $i, j, k\ge 1$ then leaving the loop the string will be, 

$$
\dashv \, \$x^*a^{i-1}x^*b^{j-1}x^*c^{k-1}
$$

and the overall length of the string will not be changed. 

The termination condition is $i=j=k=0$,

$$
\dashv \, \$x^+
$$

If some but not all of $i, j$ and $k$ are zero, the machine rejects.

**Establishing the invariant**

The case of an empty string is handled. The string is matched against the Regular Expression 
$a^+b^+c^+$ and rejected if it is not of that form. 
Then replacements are made to establish the loop invariant.

**Loop Condition**

Accept if the string matches the Regular Expression $\$x^+$; else enter the loop body.

**Loop Body**

Scan and change the first $a$, first $b$ and first $c$ to $x$; reject if this cannot be accomplished.



In [2]:
# a^i b^i c^i

tm_aibici = """# a^i b^i c^i
# mar 6, 2025 -bjr

# stage1 sets the precondition for the loop
# stage2 tests the loop condition
# stage3 maintains the loop invariant and reduces the problem by one

# built for comfort, not for speed

accept: A
reject: R
start: q0
    

# create the loop invariant

state: q0
    : : N stage1

state: stage1
    _ _ n A # accept empty string
    a $ r q1
    
state: q1
    a : r q1
    b x r q2
    
state: q2
    b : r q2
    c x r q3
    
state: q3
    c : r q3
    _ : l q4
    
state: q4
    : : l q4
    $ : N stage2


# given the loop invariant, check if done

state: stage2
    : : N r1
    
state: r1
    $ : r r1
    x : r r1
    _ : n A   # the tape is $x+ so accept
    : : l r2  # anything other than $, x or _, rewind

state: r2
    : : l r2
    $ : n stage3


# given the loop invariant, and we are not done, reduce the problem
# while maintaining the loop invariant

state: stage3
    : : n s1
    
state: s1     # find an a; for a string in the language there must be at least one
    $ : r s1
    x : r s1
    a x r s2
    
state: s2     # find a b; for a string in the language there must be at least one
    : : r s2  
    b x r s3
    _ : N R   # must see a b before the _, nothing else matters

state: s3     # find a c; for a string in the language there must be at least one
    : : r s3
    c x l s4
    _ : N R

state: s4      # the loop invariant is true; rewind and go to test (stage2)
    : : l s4
    $ : n stage2

"""

In [3]:
verbose = False
max_steps = 400

tm = MachineParser.create_from_description(tm_aibici)
for w in ['','abc','aabbcc', 'aaabbbccc', 'aaaaaaabbbbbbbccccccc']:
    print(f'\nstring: |{w}|')
    res = tm.compute_tm(w, max_steps, verbose=verbose)
    print(f'accept: {res}\n\t{tm.result_reasons[tm.result]}')
    
for w in ['ba', 'cb', 'acb', 'a','ab','ac', 'aabcc', 'abbcc', 'aabbc' ]:
    print(f'\nstring: |{w}|')
    res = tm.compute_tm(w, max_steps, verbose=verbose)
    print(f'accept: {res}\n\t{tm.result_reasons[tm.result]}')


string: ||
1 [stage1]	[_]
accept: True
	ok

string: |abc|
1 [stage1]	[a]bc_
8 [stage2]	[$]xx_
9 [r1]	[$]xx_
accept: True
	ok

string: |aabbcc|
1 [stage1]	[a]abbcc_
14 [stage2]	[$]axbxc_
15 [r1]	[$]axbxc_
31 [r1]	[$]xxxxx_
accept: True
	ok

string: |aaabbbccc|
1 [stage1]	[a]aabbbccc_
20 [stage2]	[$]aaxbbxcc_
21 [r1]	[$]aaxbbxcc_
41 [r1]	[$]xaxxbxxc_
65 [r1]	[$]xxxxxxxx_
accept: True
	ok

string: |aaaaaaabbbbbbbccccccc|
1 [stage1]	[a]aaaaaabbbbbbbccccccc_
44 [stage2]	[$]aaaaaaxbbbbbbxcccccc_
45 [r1]	[$]aaaaaaxbbbbbbxcccccc_
81 [r1]	[$]xaaaaaxxbbbbbxxccccc_
121 [r1]	[$]xxaaaaxxxbbbbxxxcccc_
165 [r1]	[$]xxxaaaxxxxbbbxxxxccc_
213 [r1]	[$]xxxxaaxxxxxbbxxxxxcc_
265 [r1]	[$]xxxxxaxxxxxxbxxxxxxc_
321 [r1]	[$]xxxxxxxxxxxxxxxxxxxx_
accept: True
	ok

string: |ba|
1 [stage1]	[b]a_
accept: False
	transition missing

string: |cb|
1 [stage1]	[c]b_
accept: False
	transition missing

string: |acb|
1 [stage1]	[a]cb_
accept: False
	transition missing

string: |a|
1 [stage1]	[a]_
accept: False
	transition

In [4]:
# a^i b^i c^i

tm_aibici_silent = """# a^i b^i c^i
# mar 6, 2025 -bjr
# mar 19, 2025 -bjr; completely silent run

# stage1 sets the precondition for the loop
# stage2 tests the loop condition
# stage3 maintains the loop invariant and reduces the problem by one

# built for comfort, not for speed

accept: A
reject: R
start: q0
    

# create the loop invariant

state: q0
    : : n stage1

state: stage1
    _ _ n A # accept empty string
    a $ r q1
    
state: q1
    a : r q1
    b x r q2
    
state: q2
    b : r q2
    c x r q3
    
state: q3
    c : r q3
    _ : l q4
    
state: q4
    : : l q4
    $ : n stage2


# given the loop invariant, check if done

state: stage2
    : : n r1
    
state: r1
    $ : r r1
    x : r r1
    _ : n A   # the tape is $x+ so accept
    : : l r2  # anything other than $, x or _, rewind

state: r2
    : : l r2
    $ : n stage3


# given the loop invariant, and we are not done, reduce the problem
# while maintaining the loop invariant

state: stage3
    : : n s1
    
state: s1     # find an a; for a string in the language there must be at least one
    $ : r s1
    x : r s1
    a x r s2
    
state: s2     # find a b; for a string in the language there must be at least one
    : : r s2  
    b x r s3
    _ : n R   # must see a b before the _, nothing else matters

state: s3     # find a c; for a string in the language there must be at least one
    : : r s3
    c x l s4
    _ : n R

state: s4      # the loop invariant is true; rewind and go to test (stage2)
    : : l s4
    $ : n stage2

"""

In [10]:
def enum_string(n,s,head):
    if n==0:
        yield ''
        return
    if n==1:
        for c in s:
            yield head+c
    else:
        for c in s:
            for q in enum_string(n-1,s,head+c):
                yield q

verbose = False
n = 13 
max_steps = (n+15)**2

tm = MachineParser.create_from_description(tm_aibici_silent)
accepted = []
for n in range(n):
    for w in enum_string(n,['a','b','c'],''):
        res = tm.compute_tm(w, max_steps, verbose=verbose)
        if res:
            accepted.append(w)
print(accepted)

['', 'abc', 'aabbcc', 'aaabbbccc', 'aaaabbbbcccc']
